In [1]:
import cv2
import numpy as np
from tkinter import filedialog
from tkinter import *
import shutil
btn_down = False

def get_points(im):
    # Set up data to send to mouse handler
    data = {}
    data['im'] = im.copy()
    data['lines'] = []

    # Set the callback function for any mouse event
    cv2.imshow("Image", im)
    cv2.setMouseCallback("Image", mouse_handler, data)
    cv2.waitKey(0)

    # Convert array to np.array in shape n,2,2
    points = np.uint16(data['lines'])

    return points, data['im']

def mouse_handler(event, x, y, flags, data):
    global btn_down

    if event == cv2.EVENT_LBUTTONUP and btn_down:
        #if you release the button, finish the line
        btn_down = False
        data['lines'][0].append((x, y)) #append the seconf point
        cv2.circle(data['im'], (x, y), 3, (0, 0, 255),5)
        cv2.line(data['im'], data['lines'][0][0], data['lines'][0][1], (0,0,255), 2)
        cv2.rectangle(data['im'], data['lines'][0][0], data['lines'][0][1], (0,0,255), 2)
        cv2.imshow("Image", data['im'])

    elif event == cv2.EVENT_MOUSEMOVE and btn_down:
        #thi is just for a ine visualization
        image = data['im'].copy()
        cv2.line(image, data['lines'][0][0], (x, y), (0,0,0), 1)
        cv2.rectangle(image, data['lines'][0][0], (x, y), (0,0,255), 2)
        cv2.imshow("Image", image)

    elif event == cv2.EVENT_LBUTTONDOWN: #and len(data['lines']) < 10:
        btn_down = True
        data['lines'].insert(0,[(x, y)]) #prepend the point
        cv2.circle(data['im'], (x, y), 3, (0, 0, 255), 5, 16)
        cv2.imshow("Image", data['im'])


# Running the code
mode="train"
filename="E:\geoshapes\TOLUCA\imagenes1\zona_4_0_0.png"
root=Tk()
root.filename =  filedialog.askopenfilename(initialdir = r"Users\ASUS\Desktop\zerodayssoftware\deteccion de terrenos",title = "Select file",filetypes = (("image",["*.png","*.jpg","*.jpeg","*.tiff","*.txt"]),("all files","*.*")),multiple=True)
filenames=root.filename
filenames
root.destroy()
for filename in filenames:
    img = cv2.imread(filename, 1)
    h,w,_=img.shape
    print(h,w)
    if w>h:
        img=cv2.resize(img,(int((h*1000)/w),1000))
    else:
        img=cv2.resize(img,(1000,int(w*1000/h)))
    h,w,_=img.shape
    pts, final_image = get_points(img)
    cv2.imshow('Image', final_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    coordenadas=[]
    yolo_F=[]   
    for rec in pts:
        x1=np.min([rec[0][0],rec[1][0]])
        y1=np.min([rec[0][1],rec[1][1]])
        x2=np.max([rec[0][0],rec[1][0]])
        y2=np.max([rec[0][1],rec[1][1]])
        coordenadas.append('{"label": "casas", "coordinates": {"x":'+str(float(x1*2))+', "y": '+str(float(y1*2))+', "width": '+str((x2*2)-(x1*2))+', "height": '+str((y2*2)-(y1*2))+'}}')
        v1=(x1+(x2-x1)/2)/w
        v2=(y1+(y2-y1)/2)/h
        v3=(x2-x1)/w
        v4=(y2-y1)/h
        yolo_F.append(" ".join(["0",str(v1),str(v2),str(v3),str(v4)]))
    yolo_F="\n".join(yolo_F)
    coordenadas=", ".join(coordenadas)
    cve=filename.replace("\\","/").split(".")[0]
    # with open(str(cve)+".json","w+") as file:
    #         file.write('[{"image": "'+str(cve)+'.png", "verified": false, "annotations":['+coordenadas+"]}]")
    #         file.close()
    #         print("archivo creado: "+cve+".json")
    cve=cve.split("/")[-1]
    with open("C:/Users/ASUS/Inteligencia_Artificial/yolov7/"+mode+"/labels/"+str(cve)+".txt","w+") as file:
            file.write(yolo_F)
            file.close()
            print("archivo creado: C:/Users/ASUS/Inteligencia_Artificial/yolov7/"+mode+"/labels/"+str(cve)+".txt")
    # shutil.copy(filename,"C:/Users/ASUS/Inteligencia_Artificial/yolov7/"+mode+"/images/"+filename.replace("\\","/").split("/")[-1])

1042 1032
archivo creado: C:/Users/ASUS/Inteligencia_Artificial/yolov7/train/labels/1012824700000000.txt
4936 4844
archivo creado: C:/Users/ASUS/Inteligencia_Artificial/yolov7/train/labels/1013000200000000.txt
171 353
archivo creado: C:/Users/ASUS/Inteligencia_Artificial/yolov7/train/labels/1013111600000000.txt
2329 730
archivo creado: C:/Users/ASUS/Inteligencia_Artificial/yolov7/train/labels/1017505900000000.txt
1024 1024
archivo creado: C:/Users/ASUS/Inteligencia_Artificial/yolov7/train/labels/zona_4_0_0.txt


In [ ]:
import geopandas as gpd
toluca=gpd.read_file("C:/Users/ASUS/Documents/GobiernoEdomex/Agua/geoshapes/Full manzanas/Manzana_Toluca.shp")
toluca.query("zona=='04'").to_file(r"E:\geoshapes\TOLUCA\zonas/zona_4_1.shp")


## Codigo generacion de imagen apartir de mascara de capa 

In [1]:
import fiona
import rasterio
import rasterio.mask
import tqdm
import cv2
src_raster_path = r"E:\geoshapes\TOLUCA\zonas\todo1.tif"

shp_file_path = r"C:\Users\ASUS\Desktop\Manzana_Toluca_modified.shp"

import os

# with fiona.open(shp_file_path, "r") as shapefile:
#     shapes = [feature["geometry"] for feature in shapefile]
shapefile=fiona.open(shp_file_path, "r")
src=rasterio.open(src_raster_path)

for i,feature in tqdm.tqdm(enumerate(shapefile),total=len(shapefile)):
    try:
        os.mkdir( src_raster_path[:-3])
    except:
        pass
    shapes=feature["geometry"]
    shapes=[shapes,]
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    array=out_image
    four_images=[array[2],array[1],array[0],array[3]]
    stacked_images = np.stack(four_images, axis=-1)
    stacked_images=cv2.resize(stacked_images,(1000,1000))
    # cv2.imwrite(r"E:\geoshapes\TOLUCA\zonas\todo/"+feature["properties"]["cve_cat"]+".png",stacked_images)
    # cv2.waitKey()
    # cv2.destroyAllWindows()
    out_meta = src.meta

    out_meta.update({"driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform})


    with rasterio.open(r"E:\geoshapes\TOLUCA\zonas\todo/"+feature["properties"]["cve_cat"]+".tif", "w", **out_meta) as dest:
        dest.write(out_image)

  0%|          | 3/9672 [00:01<1:08:42,  2.35it/s]

In [ ]:
with fiona.open(shp_file_path, "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]
# shapefile=fiona.open(shp_file_path, "r")
# for feature in shapefile:
#     print(feature["geometry"])
shapes

In [ ]:
stacked_images

array([[[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       ...,

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]], dtype=uint8)

In [ ]:
shapefile=fiona.open(shp_file_path, "r")
for feature in shapefile:
    shepe=feature["geometry"]
[shepe,]

[{'type': 'Polygon',
  'coordinates': [[(-11097238.934081577, 2186544.741223364, 0.0),
    (-11097195.888872415, 2186616.4657913493, 0.0),
    (-11097196.221967872, 2186583.292363935, 0.0),
    (-11097197.3499598, 2186563.5596933737, 0.0),
    (-11097199.479135478, 2186542.2997223586, 0.0),
    (-11097238.934081577, 2186544.741223364, 0.0)]]}]

In [ ]:
feature["properties"]["cve_cat"]

'1012531100000000'

In [ ]:
# import cv2
array=out_image
four_images=[array[2],array[1],array[0],array[3]]
stacked_images = np.stack(four_images, axis=-1)
stacked_images=cv2.resize(stacked_images,(1000,1000))
cv2.imshow("f",stacked_images)
cv2.waitKey()
cv2.destroyAllWindows()
